In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    nrows = 100000
)


# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']

/Users/j/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cols = X.columns[X.dtypes.eq('object')]
X[cols] = X[cols].apply(pd.to_numeric, errors='coerce')

In [4]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = X.dropna(axis=1)


cross_val_score(rfc, X, Y, cv=5)

array([ 0.94725791,  0.96990301,  0.96769838,  0.96419821,  0.96294444])

### PCA 

In [5]:
# Import and run PCA model 
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca.fit(X)
X_pca = pca.transform(X)


In [6]:
# Test PCA on forest
rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, X_pca, Y, cv=10)

array([ 0.93142057,  0.9413176 ,  0.94410559,  0.94210579,  0.9414    ,
        0.9408    ,  0.93979398,  0.9394879 ,  0.9374875 ,  0.93197959])

I can reduce the # of compnents to 10 and still maintain an average accuracy over 90%.

### Remove payment amount or outstanding principal.

In [7]:
list(X)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 't

In [8]:
# Remove all columns related to payment amount and outstanding principal 
columns = ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int','last_pymnt_amnt']

X_new = X.drop(columns=columns)



In [9]:
pca = PCA(n_components = 10)
pca.fit(X)
X_pca = pca.transform(X)

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, X_new, Y, cv=10)

array([ 0.76037189,  0.76996901,  0.77062294,  0.77112289,  0.7721    ,
        0.7705    ,  0.76837684,  0.76825365,  0.76505301,  0.7613284 ])

Taking out the Payment and Principal columns doesn't allow me to get above 90% accuracy when keeping the dimensional components at 10. 